## Fine-Tuning GPT Models - A Python SDK Experience

Learn how to fine-tune the <code>gpt-35-turbo-0613</code> model using Python Programming Language - An SDK / Code Experience. This notebook is based on the MS Learn tutorial [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/fine-tune?tabs=python%2Cbash).

He Zhang, Feb. 2024

### Prerequisites

* Learn the [what, why, and when to use fine-tuning.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/fine-tuning-considerations)
* An Azure subscription.
* Access to Azure OpenAI Service.
* An Azure OpenAI resource created in the supported fine-tuning region (e.g. Sweden Central).
* Prepare Training and Validation datasets:
  * at least 50 high-quality samples (preferably 1,000s) are required.
  * must be formatted in the JSON Lines (JSONL) document with UTF-8 encoding.
  * for this test notebook, we use only 10 samples for the demo purpose. 
* Python version at least: <code>3.7.1</code>
* Python libraries: <code>json, requests, os, tiktoken, time, python-dotenv, numpy, openai</code>
* The OpenAI Python library version for this test notebook: <code>0.28.1</code>
* [Jupyter Notebooks](https://jupyter.org/)

### Step 1: Setup

#### Retrieve the Azure OpenAI API key and endpoint.

Go to your resource in the Azure portal. The Endpoint and Keys can be found in the Resource Management section Go to your resource in the Azure portal.  
<img src="../../images/screenshot-aoai-keys-and-endpoint.png" alt="Screenshot of the Azure OpenAI resource management pane." width="800"/>

#### Configure credentials

Copy the <code>Endpoint</code> and access <code>KEY</code> (you can use either <code>KEY 1</code> or <code>KEY 2</code>), and paste them accordingly to the variables in the file <code>azure.env</code>. Save the file and close it. **Do not** distribute this file as this contains credential information! 
<img src="../../images/screenshot-azure-env-file.png" alt="Screenshot of the azure.env file that contains credential information - do not show it to others!" width="800"/>

#### Install required Python libraries (if not done yet)

In [11]:
%pip install openai==0.28.1  
%pip install tiktoken
%pip install numpy
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
Note: you may need to restart the kernel to use updated packages.


#### Import required Python libraries 

In [2]:
import os
import json
import time
import openai
import requests
import tiktoken
import numpy as np

from dotenv import load_dotenv

#### Load Azure OpenAI credentials

In [3]:
load_dotenv("azure.env")

openai.api_type = "azure"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_version = "2023-12-01-preview" # This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002

### Step 2: Prepare Training & Validation Datasets

#### The training and validation datasets have been made ready for you.

<code>training_set.jsonl</code>

```yaml
{"messages": 
    [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, 
     {"role": "user", "content": "Who discovered Antarctica?"}, 
     {"role": "assistant", "content": "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}]}

{"messages": 
    [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."},
     {"role": "user", "content": "What is the biggest ocean?"}, 
     {"role": "assistant", "content": "The Pacific Ocean. It's not like it's a small pond or anything."}]}

{"messages": ...}
```

<code>validation_set.jsonl</code>

```yaml
{"messages": 
    [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."}, 
     {"role": "user", "content": "What's the capital of Australia?"}, 
     {"role": "assistant", "content": "It's Canberra, not Sydney. Shocking, I know!"}]}

{"messages": 
    [{"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."},
     {"role": "user", "content": "Who wrote 'The Great Gatsby'?"}, 
     {"role": "assistant", "content": "Some guy named F. Scott Fitzgerald. Ever heard of him?"}]}
     
{"messages": ...}
```

#### Do initial checks

In [14]:
# Load the training set
with open("./data/training_set.jsonl", "r", encoding="utf-8") as f:
    training_dataset = [json.loads(line) for line in f]

# Training dataset stats
print("Number of examples in training set:", len(training_dataset))
print("First example in training set:")
for message in training_dataset[0]["messages"]:
    print(message)

Number of examples in training set: 10
First example in training set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': 'Who discovered Antarctica?'}
{'role': 'assistant', 'content': "Some chaps named Fabian Gottlieb von Bellingshausen and Mikhail Lazarev, as if they don't teach that in every school!"}


In [15]:
# Load the validation set
with open("./data/validation_set.jsonl", "r", encoding="utf-8") as f:
    validation_dataset = [json.loads(line) for line in f]

# Validation dataset stats
print("\nNumber of examples in validation set:", len(validation_dataset))
print("First example in validation set:")
for message in validation_dataset[0]["messages"]:
    print(message)


Number of examples in validation set: 10
First example in validation set:
{'role': 'system', 'content': 'Clippy is a factual chatbot that is also sarcastic.'}
{'role': 'user', 'content': "What's the capital of Australia?"}
{'role': 'assistant', 'content': "It's Canberra, not Sydney. Shocking, I know!"}


### Step 3: Upload Datasets for Fine-Tuning

In [16]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.
training_file_name = "./data/training_set.jsonl"
validation_file_name = "./data/validation_set.jsonl"

training_response = openai.File.create(
    file=open(training_file_name, "rb"), 
    purpose="fine-tune", 
    user_provided_filename=training_file_name
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name, "rb"), 
    purpose="fine-tune", 
    user_provided_filename=validation_file_name
)
validation_file_id = validation_response["id"]

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-5c8ea20003e84975a4ce476e9e9d5dad
Validation file ID: file-b85d8503ca544b2596051eab630fc525


### Step 4: Begin Fine-Tuning Job

Now you can submit your fine-tuning training job. 

The fine-tuning job will take some time to start and complete.

You can use the job ID to monitor the status of the fine-tuning job. 

In [17]:
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0613", # must be exactly this name
)

job_id = response["id"]

print("Job ID:", response["id"])
print("Status:", response["status"])
print(response)

Job ID: ftjob-1e450650d6b34aaeac7d7d55c6b94f3e
Status: pending
{
  "hyperparameters": {
    "n_epochs": -1,
    "batch_size": -1,
    "learning_rate_multiplier": 1
  },
  "status": "pending",
  "model": "gpt-35-turbo-0613",
  "training_file": "file-5c8ea20003e84975a4ce476e9e9d5dad",
  "validation_file": "file-b85d8503ca544b2596051eab630fc525",
  "id": "ftjob-1e450650d6b34aaeac7d7d55c6b94f3e",
  "created_at": 1721200845,
  "updated_at": 1721200845,
  "object": "fine_tuning.job"
}


### Step 5: Track Fine-Tuning Job Status

You can track the training job status by running:

In [ ]:
from IPython.display import clear_output
# Track fine-tuning job training status
start_time = time.time()

# Get the status of our fine-tuning job.
response = openai.FineTuningJob.retrieve(job_id)

status = response["status"]

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)
    
    response = openai.FineTuningJob.retrieve(job_id)
    print(response)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response["status"]
    print(f"Status: {status}")
    clear_output(wait=True)

print(f"Fine-tuning job {job_id} finished with status: {status}")

# List all fine-tuning jobs for this resource.
print("Checking other fine-tune jobs for this resource.")
response = openai.FineTuningJob.list()
print(f'Found {len(response["data"])} fine-tune jobs.')

To get the full results, you can run the following:

In [ ]:
# Retrieve fine_tuned_model name
response = openai.FineTuningJob.retrieve(job_id)
print(response)

fine_tuned_model = response["fine_tuned_model"]

### Step 6: Deploy The Fine-Tuned Model

Model deployment must be done using the [REST API](https://learn.microsoft.com/en-us/rest/api/cognitiveservices/accountmanagement/deployments/create-or-update?view=rest-cognitiveservices-accountmanagement-2023-05-01&tabs=HTTP), which requires separate authorization, a different API path, and a different API version.

<table>
<thead>
<tr>
<th>variable</th>
<th>Definition</th>
</tr>
</thead>
<tbody>
<tr>
<td>token</td>
<td>There are multiple ways to generate an authorization token. The easiest method for initial testing is to launch the Cloud Shell from the <a href="https://portal.azure.com" data-linktype="external">Azure portal</a>. Then run <a href="/en-us/cli/azure/account#az-account-get-access-token()" data-linktype="absolute-path"><code>az account get-access-token</code></a>. You can use this token as your temporary authorization token for API testing. We recommend storing this in a new environment variable</td>
</tr>
<tr>
<td>subscription</td>
<td>The subscription ID for the associated Azure OpenAI resource</td>
</tr>
<tr>
<td>resource_group</td>
<td>The resource group name for your Azure OpenAI resource</td>
</tr>
<tr>
<td>resource_name</td>
<td>The Azure OpenAI resource name</td>
</tr>
<tr>
<td>model_deployment_name</td>
<td>The custom name for your new fine-tuned model deployment. This is the name that will be referenced in your code when making chat completion calls.</td>
</tr>
<tr>
<td>fine_tuned_model</td>
<td>Retrieve this value from your fine-tuning job results in the previous step. It will look like <code>gpt-35-turbo-0613.ft-b044a9d3cf9c4228b5d393567f693b83</code>. You will need to add that value to the deploy_data json.</td>
</tr>
</tbody>
</table>

In [ ]:
token= os.getenv("TEMP_AUTH_TOKEN") 
subscription = "<YOUR_SUBSCRIPTION_ID>"  
resource_group = "<YOUR_RESOURCE_GROUP_NAME>"
resource_name = "<YOUR_AZURE_OPENAI_RESOURCE_NAME>"
model_deployment_name ="YOUR_CUSTOM_MODEL_DEPLOYMENT_NAME" 

deploy_params = {"api-version": "2023-05-01"} 
deploy_headers = {"Authorization": "Bearer {}".format(token), "Content-Type": "application/json"}
deploy_data = {
    "sku": {"name": "standard", "capacity": 1}, 
    "properties": {
        "model": {
            "format": "OpenAI",
            "name": "<YOUR_FINE_TUNED_MODEL>", #retrieve this value from the previous call, it will look like gpt-35-turbo-0613.ft-b044a9d3cf9c4228b5d393567f693b83
            "version": "1"
        }
    }
}
deploy_data = json.dumps(deploy_data)

print("Creating a new deployment...")
request_url = f"https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}"
r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

print(r)
print(r.reason)
print(r.json())

You can check on your deployment progress in the Azure OpenAI Studio:

<img src="../../images/screenshot-deployed-fine-tuned-model-via-sdk.png" alt="Screenshot of the Azure OpenAI Studio - showing the model deployment status." width="800"/>

### Step 7: Test And Use The Deployed Fine-Tuned Model

After your fine-tuned model is deployed, you can use it like any other deployed model in either the [Chat Playground of Azure OpenAI Studio](https://oai.azure.com/), or via the chat completion API. 

For example, you can send a chat completion call to your deployed model, as shown in the following Python code snippet. 

In [7]:
import os
import openai

openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") 
openai.api_version = "2023-05-15"
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")

response = openai.ChatCompletion.create(
    engine="gpt-35-turbo-finetune", # engine = "Custom deployment name you chose for your fine-tuning model"
    messages=[
        {"role": "system", "content": "Clippy is a factual chatbot that is also sarcastic."},
        {"role": "user", "content": "What is capital of Malaysia?"}
    ]
)

print(response)
print(response['choices'][0]['message']['content'])

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The capital of Malaysia is Kuala Lumpur.",
        "role": "assistant"
      }
    }
  ],
  "created": 1721201939,
  "id": "chatcmpl-9ltXXridBt6kkFPiGefDssQBU3KuR",
  "model": "gpt-35-turbo-1106.ft-5d12e820c0a04a8bab7f2658b9ae7db8--finetune",
  "object": "chat.completion",
  "system_fingerprint": "fp_56b453fc06",
  "usage": {
    "completion_tokens": 8,
    "prompt_tokens": 13,
    "total_tokens": 21
  }
}
The capital of Malaysia is Kuala Lumpur.


### Step 8: Delete The Deployment

It is **strongly recommended** that once you're done with this tutorial and have tested a few chat completion calls against your fine-tuned model, that you delete the model deployment, since the fine-tuned / customized models have an [hourly hosting cost](https://azure.microsoft.com/zh-cn/pricing/details/cognitive-services/openai-service/#pricing) associated with them once they are deployed.